<font size="6">Guided Project: Building Fast Queries on a CSV</font>

We will imagine that we own an online laptop store and want to build a way to answer a few different business questions about our inventory.

The goal of this guided project is to create a class that represents our inventory. The methods in that class will implement the queries that we want to answer about our inventory. We will also preprocess that data to make those queries run faster.

In [1]:
from csv import reader

opened_file = open('laptops.csv')
read_file = reader(opened_file)
rows = list(read_file)
header = rows[0]
rows = rows[1:]
print(header)
for i in range(5):
    print(rows[i])

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339']
['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898']
['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575']
['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537']
['8550527', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB', '256GB SSD',

<font size="5">Inventory Class</font>

Let's start by implementing the constructor. It will take the name of the CSV file as argument and then read the rows contained in self.header and self.rows.

In [2]:
class Inventory():
    def __init__(self, csv_filename):
        opened_file = open(csv_filename)
        read_file = reader(opened_file)
        rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
            
inventory = Inventory('laptops.csv')
print(inventory.header)
print(len(inventory.rows))

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
1303


<font size="5">Finding a Laptop Form the Id</font>

Throughout this project, we will make several improvements to the Inventory class.

The first thing that we will implement is a way to look up a laptop from a given identifier. In this way, when a customer comes to our store with a purchase slip, we can quickly identify the laptop to which it corresponds.

In [6]:
class Inventory():
    def __init__(self, csv_filename):
        opened_file = open(csv_filename)
        read_file = reader(opened_file)
        rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None

In [7]:
inventory = Inventory('laptops.csv')
print(inventory.get_laptop_from_id('3362737'))
print(inventory.get_laptop_from_id('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


<font size="5">Improve Id Lookups</font>

In our dataset, we only have about 1,300 laptops, so it might seem unnecessary to improve the performance of this query. However, you have to imagine that this code could be used in situations where the inventory contains millions of rows.

The idea is proceprocess the data into a dictionary where the keys are the IDs and the values the rows.

In [8]:
class Inventory():
    def __init__(self, csv_filename):
        opened_file = open(csv_filename)
        read_file = reader(opened_file)
        rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None

<font size="4">Test the code:</font>

In [9]:
inventory = Inventory('laptops.csv')
print(inventory.get_laptop_from_id_fast('3362737'))
print(inventory.get_laptop_from_id_fast('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


<font size="5">Comparing Performance</font>

Now, let's experiment to compare the performance of the two id lookup functions.

In [29]:
import time
import random

ids = []
for _ in range(10000):
    ids.append(str(random.randint(1000000,9999999)))

inventory = Inventory('laptops.csv')

total_time_no_dict = 0
for identifier in ids:
    start = time.time()
    inventory.get_laptop_from_id(identifier)
    end = time.time()
    total_time_no_dict += end - start

total_time_dict = 0
for identifier in ids:
    start = time.time()
    inventory.get_laptop_from_id_fast(identifier)
    end = time.time()
    total_time_dict += end - start

print(total_time_no_dict)
print(total_time_dict)

1.311955451965332
0.0054514408111572266


<font size="4">Analysis</font>

We got:
<br>1.311955451965332
<br>0.0054514408111572266

We can see a big improve in performance. If we divide 1.312 by 0.005 we see that the new method is about 262 times faster for this input size.

<font size="5">Two Laptop Promotion</font>

Sometimes, your store offers a promotion where you give a gift card. A customer can use the gift to buy up to two laptops. To avoid having to keep track of what was already spent, the gift card has a single time usage. 

You don't want to make a customer feel cheated, so whenever you issue a gift card, you want to make sure that there is at least one way to spend it in full.

We will write a function that, given a dollar amount, checks whether it is possible to spend precisely that amount by purchasing up to two laptops.

In [32]:
class Inventory():
    def __init__(self, csv_filename):
        opened_file = open(csv_filename)
        read_file = reader(opened_file)
        rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for row1 in self.rows:
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False

In [33]:
inventory = Inventory('laptops.csv')
print(inventory.check_promotion_dollars(1000))
print(inventory.check_promotion_dollars(442))

True
False


<font size="5">Improve Laptop Promotion</font>

Let's implement this to make our code run faster.

Since we only care about whether or not there is a solution, we can store all laptops prices in a set when we initialize the inventory. 

In [35]:
class Inventory():
    def __init__(self, csv_filename):
        opened_file = open(csv_filename)
        read_file = reader(opened_file)
        rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
        self.prices = set()
        for row in self.rows:
            self.prices.add(row[-1])
        
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for row1 in self.rows:
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        for price in self.prices:
            if dollars - price in self.prices:
                return True
        return False

<font size="4">Test the code:</font>

In [36]:
inventory = Inventory('laptops.csv')
print(inventory.check_promotion_dollars(1000))
print(inventory.check_promotion_dollars(442))

True
False


<font size="5">Comparing Performance</font>

Now, let's compare the performance of the last two functions that we wrote.

In [46]:
import time
import random

prices = []
for _ in range(100):
    prices.append(random.randint(100,5000))
    
inventory = Inventory('laptops.csv')
total_time_no_set = 0
for price in prices:
    start = time.time()
    inventory.check_promotion_dollars(price)
    end = time.time()
    total_time_no_set += end - start
    
total_time_set = 0
for price in prices:
    start = time.time()
    inventory.check_promotion_dollars_fast(price)
    end = time.time()
    total_time_set += end - start
    
print(total_time_no_set)
print(total_time_set)

1.7194602489471436
0.0008003711700439453


<font size="4">Analysis</font>

We got:
<br>1.7194602489471436
<br>0.0008003711700439453

We can see a big improve in performance. If we divide 1.7195 by 0.0008 we see that the new method is about 2149 times faster for this input size.

<font size="5">Finding Laptops within Budget</font>

In this project, we are going to leverage and extend that algorithm to help a customer find all laptops that fall within their budget.

More formally, we want to write a method that efficiently answers the query: Given a budget of D dollars, find all laptops whose price it at most D.

In [67]:
def row_price(row):
    return row[-1]

class Inventory():
    def __init__(self, csv_filename):
        opened_file = open(csv_filename)
        read_file = reader(opened_file)
        rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
        self.prices = set()
        for row in self.rows:
            self.prices.add(row[-1])
        self.rows_by_price = sorted(self.rows, key = row_price)
        
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for row1 in self.rows:
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        for price in self.prices:
            if dollars - price in self.prices:
                return True
        return False
    
    def find_first_laptop_more_expensive(self, target_price):
        range_start = 0                                   
        range_end = len(self.rows_by_price) - 1                       
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            price = self.rows_by_price[range_middle][-1]
            if price > target_price:                            
                range_end = range_middle                        
            else:                          
                range_start = range_middle + 1              
        if self.rows_by_price[range_start][-1] <= target_price:                  
            return -1                                      
        return range_start

In [68]:
inventory = Inventory('laptops.csv')
print(inventory.find_first_laptop_more_expensive(1000))
print(inventory.find_first_laptop_more_expensive(10000))

683
-1
